In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [191]:
class KNNRegression:
    def __init__ (self, X, Y, k):
        self.k = k
        self.X = X
        self.Y = Y
    
    def EuclidieanDistance (self, x1):
        if self.train == True:
            self.x2 = self.X[self.idx]
        else: 
            self.x2 = self.x_test
        distance = np.sqrt(np.sum(np.square(x1-self.x2)))
        return distance
    
    def calc_neighbors(self):
        costs = np.apply_along_axis(self.EuclidieanDistance, 1, self.X)
        sorted_costs = np.sort(costs)
        indices = np.where(costs <= sorted_costs[self.k-1])[0]
        return indices
    
    def predictions(self, indices):
        prediction = float(0)
        for i in indices:
            prediction += self.Y[i]
        prediction = prediction / indices.shape
        return float(prediction)
    
    def predict(self, x):
        self.train = False
        self.x_test = x
        indices =  self.calc_neighbors()
        print(indices)
        return self.predictions(indices)
    
    def accuracy(self, x_test_set, y_test_set):
        y_pred = []
        for i in  range(x_test_set.shape[0]):
            self.train = False
            self.x_test = x_test_set[i]
            indices =  self.calc_neighbors()
            y_pred.append(self.predictions(indices))
        ssr = np.sum((y_pred - y_test_set)**2)
        sst = np.sum((y_test_set - y_test_set.mean())**2)
        print(ssr, sst)
        self.r2_score = 1 - (ssr/sst)
        return self.r2_score
        
    def propogate(self):
        y_pred = []
        self.train = True
        for i in range(self.X.shape[0]):
            self.idx = i
            indices =  self.calc_neighbors()
            y_pred.append(self.predictions(indices))
        print("Model Trained")

In [195]:
a = np.array([[1, 1, 1], [3, 3, 3], [2, 2, 2]])
b = np.array([1, 3, 2])
a = KNNRegression(a, b, 2)
a.propogate()
a.accuracy(np.array([[3.1, 3.1, 3.1], [2.1, 2.1, 2.1]]), np.array([[3.1], [2.1]]))

Model Trained
1.04 0.5


-1.08

In [196]:
a.predict([1, 2, 2])

[0 2]


1.5

In [ ]:
class KNNClassification:
    def __init__ (self, X, Y, k):
        self.k = k
        self.X = X
        self.Y = Y
    
    def EuclidieanDistance(self, x1):
        x2 = self.X[self.idx]
        distance = np.sqrt(np.sum(np.square(x1-x2)))
        return distance
    
    def calc_neighbors(self):
        costs = np.apply_along_axis(self.EuclidieanDistance, 1, self.X)
        sorted_costs = np.sort(costs)
        indices = np.where(costs <= sorted_costs[self.k-1])[0]
        return indices
    
    def predict(self, indices):
        prediction = float(0)
        for i in indices:
            prediction += self.Y[i]
        prediction = prediction / indices.shape
        return float(prediction)
    
    def propogate(self):
        y_pred = []
        for i in range(self.X.shape[0]):
            self.idx = i
            indices =  self.calc_neighbors()
            y_pred.append(self.predict(indices))
            print(y_pred)